<br>

# SidraPy

- https://github.com/AlanTaranti/sidrapy
- https://sidrapy.readthedocs.io/en/latest


In [ ]:
#!pip install sidrapy --upgrade


In [1]:
import pandas as pd
import sidrapy

c:\Users\michel\Documents\Conda\envs\pablocarreira-lastest\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


<br>

## Exemplo


In [ ]:
# sidrapy.get_table(table_code="1419", territorial_level="1", ibge_territorial_code="all", period="last 12")


<br>

## Estimativa População


In [ ]:
def get_estimated_population(cod_ibge):
    # Get Table
    df = sidrapy.get_table(
        table_code='6579',
        territorial_level='6',
        ibge_territorial_code=cod_ibge,
        period='all',
        header='n',
    )

    # Dict
    dict_col = {
        'D1C': 'id_municipio',
        'D1N': 'municipio_nome',
        'V': 'n_habitantes',
        'D2N': 'ano',
    }

    # Remane Columns
    df.rename(dict_col, axis=1, inplace=True)

    # Select Columns
    df = df[[v for k, v in dict_col.items()]]

    # Adjust Columns
    df.sort_values(by=['ano'], inplace=True)
    df['id_municipio'] = pd.to_numeric(df['id_municipio'], errors='coerce')
    df['n_habitantes'] = pd.to_numeric(df['n_habitantes'], errors='coerce')
    df['ano'] = pd.to_numeric(df['ano'], errors='coerce')

    # Results
    return df


In [ ]:
cod_ibge = '3526902'  # Limeira

df = get_estimated_population(cod_ibge)
print(df.dtypes)
df.head()
